In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from dotmap import DotMap
from pathlib import Path
from collections.abc import Mapping, Iterable

from IPython.display import display

%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
#with open("dfs_stock.pkl", 'rb') as f:
#    dfs_stock = pickle.load(f)
#with open("dfs_emissions.pkl", 'rb') as f:
#    dfs_emissions = pickle.load(f)
#with open("dfs_env_em_intensity.pkl", 'rb') as f:
#    dfs_env_em_intensity = pickle.load(f)
with open("dfs_prod_animals.pkl", 'rb') as f:
    dfs_prod_animals = pickle.load(f)
with open("trade_matrix.pkl", 'rb') as f:
    trade_matrix = pickle.load(f)
with open("trade_matrix_live_animals.pkl", 'rb') as f:
    trade_matrix_live_animals = pickle.load(f)

In [6]:
def keep_non_redundant_info_live_animals(trade_matrix_live_animals,priority_reporter,unit):
    ## 1000 Head -> Head
    unit_filter = trade_matrix_live_animals["Unit Importer Report"] == "1000 Head"
    trade_matrix_live_animals.loc[unit_filter, "Quantity Importer Report"] = (trade_matrix_live_animals[unit_filter]
                                                                              ["Quantity Importer Report"].multiply(1000))
    trade_matrix_live_animals.loc[unit_filter, "Unit Importer Report"] = "Head"

    unit_filter = trade_matrix_live_animals["Unit Exporter Report"] == "1000 Head"
    trade_matrix_live_animals.loc[unit_filter, "Quantity Exporter Report"] = (trade_matrix_live_animals[unit_filter]
                                                                              ["Quantity Exporter Report"].multiply(1000))
    trade_matrix_live_animals.loc[unit_filter, "Unit Exporter Report"] = "Head"
    ##
    
    trade_matrix_live_animals_copy = trade_matrix_live_animals.set_index(["Year","Item","Importer","Exporter"])
    #trade_matrix_live_animals_copy.sort_index(inplace=True) #TODO ???

    # df_*reporter*_*unit* contain data from trade_matrix_live_animals that come from *reporter* and with given *unit*
    df_importer_tonnes = select_on(trade_matrix_live_animals_copy,
                                   column_filter_dict={"Unit Importer Report":["tonnes"]}
                                  )[["Unit Importer Report","Quantity Importer Report"]]
    df_importer_tonnes = df_importer_tonnes[~df_importer_tonnes.index.duplicated()]
    df_importer_tonnes.columns = ["Unit","Quantity"]

    df_importer_head = select_on(trade_matrix_live_animals_copy,
                                 column_filter_dict={"Unit Importer Report":["Head"]}
                                )[["Unit Importer Report","Quantity Importer Report"]]
    df_importer_head = df_importer_head[~df_importer_head.index.duplicated()]
    df_importer_head.columns = ["Unit","Quantity"]

    df_exporter_tonnes = select_on(trade_matrix_live_animals_copy,
                                   column_filter_dict={"Unit Exporter Report":["tonnes"]}
                                  )[["Unit Exporter Report","Quantity Exporter Report"]]
    df_exporter_tonnes = df_exporter_tonnes[~df_exporter_tonnes.index.duplicated()]
    df_exporter_tonnes.columns = ["Unit","Quantity"]

    df_exporter_head = select_on(trade_matrix_live_animals_copy,
                                 column_filter_dict={"Unit Exporter Report":["Head"]}
                                )[["Unit Exporter Report","Quantity Exporter Report"]]
    df_exporter_head = df_exporter_head[~df_exporter_head.index.duplicated()]
    df_exporter_head.columns = ["Unit","Quantity"]

    
    # combine all available data removing redundancy, giving priority to priority_reporter and then to unit
    if priority_reporter == "importer":
        if unit == "tonnes":
            res = (df_importer_tonnes
                   .combine_first(df_importer_head)
                   .combine_first(df_exporter_tonnes)
                   .combine_first(df_exporter_head))
        else:
            res = (df_importer_head
                   .combine_first(df_importer_tonnes)
                   .combine_first(df_exporter_head)
                   .combine_first(df_exporter_tonnes))
    else :
        if unit == "tonnes":
            res = (df_exporter_tonnes
                   .combine_first(df_exporter_head)
                   .combine_first(df_importer_tonnes)
                   .combine_first(df_importer_head))
        else:
            res = (df_exporter_head
                   .combine_first(df_exporter_tonnes)
                   .combine_first(df_importer_head)
                   .combine_first(df_importer_tonnes))
    
    res.reset_index(inplace=True)
    
    #TODO conversion Head or tonnes -> unit
    animal_mass_by_country = pd.read_csv("data/definitions_and_standards/TAM_Country.csv", 
                                         encoding="Latin1", 
                                         usecols=["AreaName","Livestock_Type","ParamValue","Units"])
    animal_mass_by_country = select_on(animal_mass_by_country,
                                       column_filter_dict={"Livestock_Type":["Cattle, non-dairy",
                                                                             "Swine, market",
                                                                             "Chicken, broilers"]})
    animal_mass_by_country["Livestock_Type"] = (animal_mass_by_country["Livestock_Type"]
                                                .replace({"Cattle, non-dairy":"Cattle",
                                                          "Swine, market":"Pigs",
                                                          "Chicken, broilers":"Chickens"}))
    animal_mass_by_country.columns = ["Exporter","Item","Animal Mass","Unit Animal Mass"]
    res = res.merge(animal_mass_by_country,how="left")
    res = res.set_index("Item")
    res["Animal Mass"].fillna((animal_mass_by_country[animal_mass_by_country["Exporter"].isna()]
                     .drop_duplicates()
                     .set_index("Item")["Animal Mass"]), inplace=True)
    res["Unit Animal Mass"].fillna((animal_mass_by_country[animal_mass_by_country["Exporter"].isna()]
                     .drop_duplicates()
                     .set_index("Item")["Unit Animal Mass"]), inplace=True)
    res = res.reset_index()
     
    res["Quantity (tonnes)"] = res.apply(lambda r: r["Quantity"] if r["Unit"]== "tonnes" 
                                      else r["Quantity"]*r["Animal Mass"]/1000, axis=1)
    res["Quantity (head)"] = res.apply(lambda r: r["Quantity"] if r["Unit"]== "Head" 
                                      else r["Quantity"]*1000/r["Animal Mass"], axis=1)
    return res[["Item","Year","Importer","Exporter","Quantity (tonnes)","Quantity (head)"]]

keep_non_redundant_info_live_animals(trade_matrix_live_animals,"importer","tonnes")

,Item,Year,Importer,Exporter,Quantity (tonnes),Quantity (head)
0,Cattle,Y1986,Algeria,Austria,373.800,8.900000e+02
1,Cattle,Y1986,Algeria,Bulgaria,78.200,2.000000e+02
2,Cattle,Y1986,Algeria,Denmark,168.000,4.000000e+02
3,Cattle,Y1986,Algeria,France,1110.480,2.644000e+03
4,Cattle,Y1986,Algeria,Netherlands,112.140,2.670000e+02
5,Cattle,Y1986,Andorra,France,2569.560,6.118000e+03
6,Cattle,Y1986,Andorra,Spain,1239.420,2.951000e+03
7,Cattle,Y1986,Argentina,Brazil,0.305,1.000000e+00
8,Cattle,Y1986,Argentina,Canada,11.281,2.900000e+01
9,Cattle,Y1986,Argentina,Paraguay,1.830,6.000000e+00


In [77]:
df_index = trade_matrix_live_animals.set_index(["Year","Item","Importer","Exporter"])
df_index["Unit"] = np.nan
df_index["Value"] = np.nan
df_index = df_index[["Unit", "Value"]].drop_duplicates()
df_index

,,,,Unit,Value
Year,Item,Importer,Exporter,,
Y1986,Cattle,Algeria,Austria,NaN,NaN


In [65]:
index

,Year,Item,Importer,Exporter
13,Y1986,Cattle,Algeria,Austria
14,Y1986,Chickens,Algeria,Austria
21,Y1986,Chickens,Algeria,Belgium-Luxembourg
49,Y1986,Cattle,Algeria,Bulgaria
79,Y1986,Cattle,Algeria,Denmark
80,Y1986,Chickens,Algeria,Denmark
91,Y1986,Cattle,Algeria,France
93,Y1986,Chickens,Algeria,France
136,Y1986,Chickens,Algeria,Hungary
178,Y1986,Cattle,Algeria,Netherlands


In [56]:
trade_matrix_live_animals.groupby(by=["Importer","Exporter","Item","Year"]).count()["Unit Importer Report"].unique()

array([0, 4, 1, 2], dtype=int64)

In [57]:
trade_matrix_live_animals.groupby(by=["Importer","Exporter","Item","Year"]).count()["Unit Exporter Report"].unique()

array([1, 4, 0, 2], dtype=int64)

In [59]:
trade_matrix_live_animals.set_index(by=["Year","Item","Importer","Exporter"]).count()

Importer Code  \
Year  Item   Importer                           Exporter                                   
Y1986 Cattle Algeria                            Austria                                1   
                                                Bulgaria                               1   
                                                Denmark                                1   
                                                France                                 1   
                                                Netherlands                            1   
             Andorra                            France                                 1   
                                                Spain                                  1   
             Argentina                          Brazil                                 1   
                                                Canada                                 1   
                                                Paraguay                               1   
                                                United States of America               1   
                                                Uruguay                                1   
             Australia                          Belgium-Luxembourg                     1   
                                                Japan                                  1   
                                                Netherlands                            1   
                                                New Zealand                            1   
                                                United Kingdom                         1   
                                                United States of America               1   
             Austria                            Czechoslovakia                         1   
                                                Hungary                                1   
                                                Poland                                 1   
                                                United Kingdom                         1   
             Bahrain                            Australia                              1   
                                                Kenya                                  1   
                                                Netherlands                            1   
             Barbados                           United States of America               1   
             Belgium-Luxembourg                 Australia                              1   
                                                Austria                                1   
                                                Canada                                 1   
                                                France                                 1   
...                                                                                  ...   
Y2017 Pigs   United Kingdom                     Denmark                                4   
                                                France                                 2   
                                                Ireland                                4   
                                                Luxembourg                             2   
                                                Netherlands                            2   
                                                Spain                                  2   
                                                United States of America               4   
             United Republic of Tanzania        Kenya                                  4   
                                                Lebanon                                2   
                                                South Africa                           4   
                                                Zambia                                 2   
             United States 

In [5]:
def keep_non_redundant_info(trade_matrix,priority_reporter):
    """
        Replace columns ["Unit Importer Report","Quantity Importer Report","Unit Exporter Report","Quantity Exporter Report"]
        by columns ["Unit","Quantity"] removing redundance giving priority to info from priority_reporter (in {"importer","exporter"})
    """
    if priority_reporter == "importer":
        trade_matrix[["Unit","Quantity"]] = trade_matrix[["Unit Importer Report","Quantity Importer Report"]]
        trade_matrix["Unit"].fillna(trade_matrix["Unit Exporter Report"],inplace=True)
        trade_matrix["Quantity"].fillna(trade_matrix["Quantity Exporter Report"],inplace=True)
    else:
        trade_matrix[["Unit","Quantity"]] = trade_matrix[["Unit Exporter Report","Quantity Exporter Report"]]
        trade_matrix["Unit"].fillna(trade_matrix["Unit Importer Report"],inplace=True)
        trade_matrix["Quantity"].fillna(trade_matrix["Quantity Importer Report"],inplace=True)
    trade_matrix.drop(columns=["Unit Importer Report","Quantity Importer Report","Unit Exporter Report","Quantity Exporter Report"],inplace=True)

#### Import Export Products

In [6]:
keep_non_redundant_info(trade_matrix,"importer")

In [7]:
trade_matrix["Year"] = trade_matrix["Year"].apply(lambda str : str[1:])

trade_matrix["Equivalent Quantity (tonnes)"] = trade_matrix["Factor"] * trade_matrix["Quantity"]

item_corresp = {'Milk Equivalent':'Milk, whole fresh cow', 
                'Eggs, equivalent':'Eggs, hen, in shell', 
                'Bovine Meat':'Meat, cattle', 
                'Pigmeat':'Meat, pig', 
                'Chicken Meat':'Meat, chicken'}

trade_matrix.replace(to_replace=item_corresp, inplace=True)
detail_import = trade_matrix.groupby(by=["Item Group","Importer","Year","Exporter"]).sum()[["Equivalent Quantity (tonnes)"]].reset_index()
total_export = detail_import.groupby(by=["Item Group","Year","Exporter"]).sum().reset_index()

#### Production Meat

In [10]:
trade_matrix_live_animals["Unit Importer Report"].unique()

trade_matrix_live_animals["Unit Exporter Report"].unique()

array(['Head', '1000 Head', nan, 'tonnes'], dtype=object)

In [13]:
trade_matrix_live_animals.sort_values(by=["Importer","Year","Exporter","Item"])

,Importer Code,Importer,Exporter Code,Exporter,Item Code,Item,Year,Unit Importer Report,Quantity Importer Report,Unit Exporter Report,Quantity Exporter Report
8935640,2,Afghanistan,100,India,1057,Chickens,Y1986,NaN,NaN,1000 Head,66.0
8997275,2,Afghanistan,100,India,1057,Chickens,Y1987,NaN,NaN,1000 Head,26.0
9060934,2,Afghanistan,100,India,1057,Chickens,Y1988,NaN,NaN,1000 Head,20.0
9197971,2,Afghanistan,100,India,1057,Chickens,Y1990,NaN,NaN,1000 Head,8.0
9271851,2,Afghanistan,100,India,1057,Chickens,Y1991,NaN,NaN,1000 Head,33.0
9548224,2,Afghanistan,150,Netherlands,1034,Pigs,Y1994,NaN,NaN,Head,162.0
10256615,2,Afghanistan,165,Pakistan,866,Cattle,Y2000,NaN,NaN,Head,2598.0
10384478,2,Afghanistan,165,Pakistan,866,Cattle,Y2001,NaN,NaN,Head,3601.0
10384481,2,Afghanistan,165,Pakistan,1057,Chickens,Y2001,NaN,NaN,1000 Head,1.0
10480064,2,Afghanistan,102,Iran (Islamic Republic of),1057,Chickens,Y2002,NaN,NaN,1000 Head,1.0


In [371]:
df_meat_prod = select_on(dfs_prod_animals["production"],
                  column_filter_dict={"Item" : ['Meat, cattle',
                                                'Meat, chicken',  
                                                'Meat, pig'],
                                      "Unit":["tonnes"]})

df_meat_prod = df_meat_prod[["Item","Area","Year","Area","Value"]]
df_meat_prod.columns = ["Item","Consumer","Year","Producer","Quantity (tonnes)"]

df_meat_cons = select_on(detail_import,
                        column_filter_dict={"Item Group":['Meat, cattle',
                                                'Meat, chicken',  
                                                'Meat, pig']})
df_meat_cons.columns = ["Item","Consumer","Year","Producer","Quantity (tonnes)"]

df_meat_cons = pd.concat([df_meat_cons, df_meat_prod])
#df_meat_cons.sort_values(by=["Consumer","Item","Year","Quantity (tonnes)"])

In [372]:
df_yield_meat = select_on(dfs_prod_animals["yield/carcass_weight"],
                                 column_filter_dict={"Item":['Meat, cattle',
                                                             'Meat, chicken',
                                                             'Meat, pig']})
df_yield_meat = df_yield_meat[["Item","Year","Area","Value"]]
df_yield_meat.columns = ["Item","Year","Producer","Yield"]

In [373]:
total_export.columns = ["Item","Year","Consumer","Export Quantity (tonnes)"]
df_meat_cons = df_meat_cons.merge(total_export)

df_sum = df_meat_cons.groupby(by=["Item","Consumer","Year"]).sum()["Quantity (tonnes)"].reset_index()
df_sum.columns = ["Item","Consumer","Year","Total (all producers)"]


df_meat_cons = df_meat_cons.merge(df_sum)
df_meat_cons["Consumed Quantity (tonnes)"] = df_meat_cons["Quantity (tonnes)"] - df_meat_cons["Export Quantity (tonnes)"]*df_meat_cons["Quantity (tonnes)"]/df_meat_cons["Total (all producers)"]

In [374]:
df_meat_cons['Year']=df_meat_cons['Year'].astype(int)
df_yield_meat['Year']=df_yield_meat['Year'].astype(int)

df_meat_cons = df_meat_cons.merge(df_yield_meat)
df_meat_cons["Animal Number"] = 1000 * df_meat_cons["Consumed Quantity (tonnes)"] / df_meat_cons["Yield"]

In [ ]:
#TODO delete cell ?
df_meat_animal = select_on(dfs_prod_animals["production"],
                  column_filter_dict={"Item" : ['Meat indigenous, cattle',
                                                'Meat indigenous, chicken',  
                                                'Meat indigenous, pig'],
                                      "Unit":["Head"]})#.groupby(by=["Area","Item","Year"]).sum()

## Import Export Live Animals

#### Production Egg or Milk

In [ ]:
df_egg_milk_prod = select_on(dfs_prod_animals["production"],
                  column_filter_dict={"Item" : ['Eggs, hen, in shell', 'Milk, whole fresh cow'],
                                      "Unit":["tonnes"]})

df_egg_milk_prod = df_egg_milk_prod[["Item","Area","Year","Area","Value"]]
df_egg_milk_prod.columns = ["Item","Consumer","Year","Producer","Quantity (tonnes)"]

df_egg_milk_cons = select_on(detail_import,
                        column_filter_dict={"Item Group":['Eggs, hen, in shell',
                                                'Milk, whole fresh cow']})
df_egg_milk_cons.columns = ["Item","Consumer","Year","Producer","Quantity (tonnes)"]

df_egg_milk_cons = pd.concat([df_egg_milk_cons, df_egg_milk_prod])



df_yield_egg_milk = select_on(dfs_prod_animals["yield"],
                                 column_filter_dict={"Item":['Eggs, hen, in shell',
                                                             'Milk, whole fresh cow']})
df_yield_egg_milk = df_yield_egg_milk[["Item","Year","Area","Value"]]
df_yield_egg_milk.columns = ["Item","Year","Producer","Yield"]


total_export.columns = ["Item","Year","Consumer","Export Quantity (tonnes)"]
df_egg_milk_cons = df_egg_milk_cons.merge(total_export)

df_sum = df_egg_milk_cons.groupby(by=["Item","Consumer","Year"]).sum()["Quantity (tonnes)"].reset_index()
df_sum.columns = ["Item","Consumer","Year","Total (all producers)"]

df_egg_milk_cons = df_egg_milk_cons.merge(df_sum)
df_egg_milk_cons["Consumed Quantity (tonnes)"] = (df_egg_milk_cons["Quantity (tonnes)"] 
                                                  - df_egg_milk_cons["Export Quantity (tonnes)"]
                                                          *df_egg_milk_cons["Quantity (tonnes)"]/
                                                          df_egg_milk_cons["Total (all producers)"])



df_egg_milk_cons['Year']=df_egg_milk_cons['Year'].astype(int)
df_yield_egg_milk['Year']=df_yield_egg_milk['Year'].astype(int)

df_egg_milk_cons = df_egg_milk_cons.merge(df_yield_egg_milk)
df_egg_milk_cons["Animal Number"] = 1000 * df_egg_milk_cons["Consumed Quantity (tonnes)"] / df_egg_milk_cons["Yield"]
df_egg_milk_cons.sort_values(by=["Item","Consumer","Year","Animal Number"])

In [102]:
#TODO delete cell ?
df_laying_hen = select_on(dfs_prod_animals["laying"],column_filter_dict={"Item" : ['Eggs, hen, in shell']})#.groupby(by=["Area","Item","Year"]).sum()
df_dairy_cow = select_on(dfs_prod_animals["milk_animals"],column_filter_dict={"Item" : ['Milk, whole fresh cow']})#.groupby(by=["Area","Item","Year"]).sum()

## explorations

In [22]:
df_mass_animal = pd.read_csv("data/detailed_trade_matrix/TAM_Country.csv", encoding="latin1")
df_mass_animal["Livestock_Type"].unique()

array(['Cattle, dairy', 'Cattle, non-dairy', 'Buffalo', 'Swine, market',
       'Swine, breeding', 'Chicken, layers', 'Chicken, broilers', 'Ducks',
       'Turkeys', 'Sheep', 'Goats', 'Horses', 'Asses', 'Mules', 'Camels',
       'Llamas'], dtype=object)

In [152]:
print(dfs_prod_animals.keys())

odict_keys(['laying', 'production', 'producing_animals/slaughtered', 'milk_animals', 'prod_popultn', 'yield', 'yield/carcass_weight'])


In [128]:
print(dfs_prod_animals["laying"]["Item"].unique())
dfs_prod_animals["laying"].head()

['Eggs, hen, in shell' 'Eggs, other bird, in shell'
 'Eggs, hen, in shell (number)' 'Eggs Primary']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
0,2,Afghanistan,1062,"Eggs, hen, in shell",5313,1961,Head,4000000.0,FAO estimate
1,2,Afghanistan,1062,"Eggs, hen, in shell",5313,1962,Head,4400000.0,FAO estimate
2,2,Afghanistan,1062,"Eggs, hen, in shell",5313,1963,Head,4600000.0,FAO estimate
3,2,Afghanistan,1062,"Eggs, hen, in shell",5313,1964,Head,4800000.0,FAO estimate
4,2,Afghanistan,1062,"Eggs, hen, in shell",5313,1965,Head,5200000.0,FAO estimate


In [5]:
trade_matrix_live_animals.head()

,Importer Code,Importer,Exporter Code,Exporter,Item Code,Item,Year,Unit Importer Report,Quantity Importer Report,Unit Exporter Report,Quantity Exporter Report
13,4,Algeria,11,Austria,866,Cattle,Y1986,Head,890.0,Head,2062.0
14,4,Algeria,11,Austria,1057,Chickens,Y1986,1000 Head,204.0,1000 Head,258.0
21,4,Algeria,15,Belgium-Luxembourg,1057,Chickens,Y1986,1000 Head,6207.0,1000 Head,6381.0
49,4,Algeria,27,Bulgaria,866,Cattle,Y1986,Head,200.0,Head,855.0
79,4,Algeria,54,Denmark,866,Cattle,Y1986,Head,400.0,Head,600.0


In [10]:
trade_matrix_live_animals["Unit Exporter Report"].unique()

array(['Head', '1000 Head', nan, 'tonnes'], dtype=object)

In [15]:
select_on(trade_matrix_live_animals, column_filter_dict={"Unit Importer Report":['Head', '1000 Head']})

,Importer Code,Importer,Exporter Code,Exporter,Item Code,Item,Year,Unit Importer Report,Quantity Importer Report,Unit Exporter Report,Quantity Exporter Report
13,4,Algeria,11,Austria,866,Cattle,Y1986,Head,890.0,Head,2062.0
14,4,Algeria,11,Austria,1057,Chickens,Y1986,1000 Head,204.0,1000 Head,258.0
21,4,Algeria,15,Belgium-Luxembourg,1057,Chickens,Y1986,1000 Head,6207.0,1000 Head,6381.0
49,4,Algeria,27,Bulgaria,866,Cattle,Y1986,Head,200.0,Head,855.0
79,4,Algeria,54,Denmark,866,Cattle,Y1986,Head,400.0,Head,600.0
80,4,Algeria,54,Denmark,1057,Chickens,Y1986,1000 Head,60.0,1000 Head,60.0
91,4,Algeria,68,France,866,Cattle,Y1986,Head,2644.0,Head,6899.0
93,4,Algeria,68,France,1057,Chickens,Y1986,1000 Head,1281.0,1000 Head,1168.0
136,4,Algeria,97,Hungary,1057,Chickens,Y1986,1000 Head,3190.0,1000 Head,2919.0
178,4,Algeria,150,Netherlands,866,Cattle,Y1986,Head,267.0,Head,911.0


In [16]:
select_on(trade_matrix_live_animals, column_filter_dict={"Unit Importer Report":['tonnes']})

,Importer Code,Importer,Exporter Code,Exporter,Item Code,Item,Year,Unit Importer Report,Quantity Importer Report,Unit Exporter Report,Quantity Exporter Report
6991970,2,Afghanistan,102,Iran (Islamic Republic of),866,Cattle,Y2014,tonnes,138.0,tonnes,138.0
6991971,2,Afghanistan,102,Iran (Islamic Republic of),866,Cattle,Y2014,tonnes,138.0,Head,458.0
6991981,2,Afghanistan,102,Iran (Islamic Republic of),1057,Chickens,Y2014,tonnes,496.0,1000 Head,8261.0
6991982,2,Afghanistan,102,Iran (Islamic Republic of),1057,Chickens,Y2014,tonnes,496.0,tonnes,496.0
6992375,2,Afghanistan,165,Pakistan,1057,Chickens,Y2014,tonnes,0.0,1000 Head,2440.0
6992376,2,Afghanistan,165,Pakistan,1057,Chickens,Y2014,tonnes,0.0,tonnes,0.0
6993006,3,Albania,11,Austria,1057,Chickens,Y2014,tonnes,21.0,1000 Head,541.0
6993007,3,Albania,11,Austria,1057,Chickens,Y2014,tonnes,21.0,tonnes,21.0
6993026,3,Albania,27,Bulgaria,866,Cattle,Y2014,tonnes,1364.0,tonnes,1359.0
6993027,3,Albania,27,Bulgaria,866,Cattle,Y2014,tonnes,1364.0,Head,5892.0


In [47]:
df = trade_matrix_live_animals.copy()
df.set_index(["Year","Item","Importer","Exporter","Unit Importer Report"], inplace=True)
df = df[["Quantity Importer Report"]]

In [50]:
df.duplicated()

Year   Item      Importer        Exporter                  Unit Importer Report
Y1986  Cattle    Algeria         Austria                   Head                     True
       Chickens  Algeria         Austria                   1000 Head                True
                                 Belgium-Luxembourg        1000 Head               False
       Cattle    Algeria         Bulgaria                  Head                     True
                                 Denmark                   Head                     True
       Chickens  Algeria         Denmark                   1000 Head                True
       Cattle    Algeria         France                    Head                     True
       Chickens  Algeria         France                    1000 Head                True
                                 Hungary                   1000 Head                True
       Cattle    Algeria         Netherlands               Head                     True
       Chickens  Algeria      

In [25]:
trade_matrix_live_animals.groupby(by=["Year","Item","Importer"]).isin()

Importer Code  Exporter Code  \
Year  Item   Importer                                                           
Y1986 Cattle Algeria                                         5              5   
             Andorra                                         2              2   
             Argentina                                       5              5   
             Australia                                       6              6   
             Austria                                         4              4   
             Bahrain                                         3              3   
             Barbados                                        1              1   
             Belgium-Luxembourg                              8              8   
             Bermuda                                         1              1   
             Bolivia (Plurinational State of)                4              4   
             Brazil                                          9              9   
             Brunei Darussalam                               1              1   
             Bulgaria                                        2              2   
             Canada                                          7              7   
             Cayman Islands                                  1              1   
             China, Hong Kong SAR                            3              3   
             China, Macao SAR                                2              2   
             China, Taiwan Province of                       4              4   
             China, mainland                                 6              6   
             Colombia                                        5              5   
             Costa Rica                                      2              2   
             Cuba                                            2              2   
             Czechoslovakia                                  6              6   
             Denmark                                         8              8   
             Djibouti                                        1              1   
             Dominican Republic                              1              1   
             Ecuador                                         6              6   
             Egypt                                          11             11   
             El Salvador                                     4              4   
             Falkland Islands (Malvinas)                     1              1   
...                                                        ...            ...   
Y2017 Pigs   Romania                                        56             56   
             Russian Federation                             16             16   
             Rwanda                                          4              4   
             Sao Tome and Principe                           2              2   
             Saudi Arabia                                    4              4   
             Serbia                                         36             36   
             Seychelles                                      2              2   
             Singapore                                       8              8   
             Slovakia                                       28             28   
             Slovenia                                       18             18   
             South Africa                                    4              4   
             Spain                                          46             46   
             Suriname                                        4              4   
             Sweden                                         12             12   
             Switzerland                                    22             22   
             Thailand                                       18             18   
             Timor-Leste                                     2              2   

In [14]:
print(len(dfs_prod_animals["production"]["Item"].unique()))
print(dfs_prod_animals["production"]["Item"].unique())
display(dfs_prod_animals["production"].head())
display(dfs_prod_animals["production"].groupby(by=["Unit","Item"]).sum())
select_on(dfs_prod_animals["production"], column_filter_dict = {"Unit":["Head"]})["Item"].unique()

66
['Eggs, hen, in shell' 'Eggs, hen, in shell (number)' 'Honey, natural'
 'Meat indigenous, camel' 'Meat indigenous, cattle'
 'Meat indigenous, chicken' 'Meat indigenous, goat'
 'Meat indigenous, sheep' 'Meat, cattle' 'Meat, game' 'Meat, goat'
 'Meat, sheep' 'Milk, whole fresh cow' 'Milk, whole fresh goat'
 'Milk, whole fresh sheep' 'Silk-worm cocoons, reelable' 'Wool, greasy'
 'Eggs, other bird, in shell' 'Eggs, other bird, in shell (number)'
 'Meat indigenous, pig' 'Meat, chicken' 'Meat, nes' 'Meat, pig'
 'Milk, whole fresh buffalo' 'Meat indigenous, horse'
 'Meat indigenous, rabbit' 'Meat indigenous, turkey' 'Meat, camel'
 'Meat, horse' 'Milk, whole fresh camel' 'Beeswax' 'Meat indigenous, duck'
 'Meat indigenous, geese' 'Skins, sheep, with wool' 'Hides, cattle, fresh'
 'Meat, duck' 'Meat, turkey' 'Meat, goose and guinea fowl' 'Meat, rabbit'
 'Skins, sheep, fresh' 'Meat indigenous, buffalo' 'Meat, buffalo'
 'Skins, goat, fresh' 'Meat indigenous, other camelids'
 'Meat indigenous, r

,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
49,2,Afghanistan,1062,"Eggs, hen, in shell",5510,1961,tonnes,10000.0,FAO estimate
50,2,Afghanistan,1062,"Eggs, hen, in shell",5510,1962,tonnes,11000.0,FAO estimate
51,2,Afghanistan,1062,"Eggs, hen, in shell",5510,1963,tonnes,11500.0,FAO estimate
52,2,Afghanistan,1062,"Eggs, hen, in shell",5510,1964,tonnes,12000.0,FAO estimate
53,2,Afghanistan,1062,"Eggs, hen, in shell",5510,1965,tonnes,12800.0,FAO estimate


Area Code  Item Code  \
Unit   Item                                                        
Head   Meat indigenous, ass                     78254     534072   
       Meat indigenous, bird nes                61957     459616   
       Meat indigenous, buffalo                198485    1545480   
       Meat indigenous, camel                  274241    2120505   
       Meat indigenous, cattle                1295508    9704320   
       Meat indigenous, chicken               1305991   11275858   
       Meat indigenous, duck                   500290    4321730   
       Meat indigenous, geese                  261978    2230467   
       Meat indigenous, goat                  1143905    9370560   
       Meat indigenous, horse                  489096    4236960   
       Meat indigenous, mule                    31535     178716   
       Meat indigenous, other camelids          10017     123066   
       Meat indigenous, pig                   1177796    9869525   
       Meat indigenous, poultry               1305640   18293150   
       Meat indigenous, rabbit                 324267    3072784   
       Meat indigenous, rodents                 10017     122324   
       Meat indigenous, sheep                 1175604    9539112   
       Meat indigenous, turkey                 399583    3554490   
No     Eggs, hen, in shell                     658480    5450184   
       Eggs, hen, in shell (number)           1298496   10935683   
       Eggs, other bird, in shell              159485    1300472   
       Eggs, other bird, in shell (number)     300097    2418780   
tonnes Beef and Buffalo Meat                  1320177   18916044   
       Beeswax                                 365053    3418870   
       Eggs Primary                           1326646   18691189   
       Eggs, hen, in shell                    1326997   11134008   
       Eggs, other bird, in shell              312805    2584579   
       Hair, horse                               9958      88000   
       Hides, buffalo, fresh                   198485    1521630   
       Hides, cattle, fresh                   1291048    9425264   
...                                               ...        ...   
       Meat, cattle                           1320528    9081825   
       Meat, chicken                          1334457   11130160   
       Meat, duck                              511718    4411763   
       Meat, game                              433331    3630886   
       Meat, goat                             1164320    9406233   
       Meat, goose and guinea fowl             283936    2390644   
       Meat, horse                             512424    4311210   
       Meat, mule                               32130     179982   
       Meat, nes                               398722    3529482   
       Meat, other camelids                     10206     125064   
       Meat, other rodents                      10206     124308   
       Meat, pig                              1203413    9893565   
       Meat, rabbit                            346318    3223325   
       Meat, sheep                            1196458    9356729   
       Meat, turkey                            412711    3678480   
       Milk, whole fresh buffalo               133971    1177338   
       Milk, whole fresh camel                 205622    1550360   
       Milk, whole fresh cow                  1249664    8784720   
       Milk, whole fresh goat                  651810    5418240   
       Milk, whole fresh sheep                 490307    3700176   
       Milk,Total                             1269887   17919260   
       Offals, nes                              88082     731709   
       Sheep and Goat Meat                    1295893   18521750   
       Silk-worm cocoons, reelable             217998    1932735   
       Skins, furs                               2725      15535   
       Skins, goat, fresh                     1131220    9226025   
       Skins, sheep, fresh      

array(['Meat indigenous, camel', 'Meat indigenous, cattle',
       'Meat indigenous, chicken', 'Meat indigenous, goat',
       'Meat indigenous, sheep', 'Meat indigenous, pig',
       'Meat indigenous, horse', 'Meat indigenous, rabbit',
       'Meat indigenous, turkey', 'Meat indigenous, duck',
       'Meat indigenous, geese', 'Meat indigenous, buffalo',
       'Meat indigenous, other camelids', 'Meat indigenous, rodents',
       'Meat indigenous, ass', 'Meat indigenous, bird nes',
       'Meat indigenous, mule', 'Meat indigenous, poultry'], dtype=object)

In [411]:
df1 = select_on(dfs_prod_animals["production"],
                  column_filter_dict={"Item" : ['Meat indigenous, cattle'
                                                #,'Meat indigenous, chicken', 'Meat indigenous, pig'
                                               ], "Unit":["tonnes"]}).sort_values(by=["Area","Year","Item"])[["Area","Year","Item","Value"]]
df1.columns = [["Area","Year","Item1","Value1"]]
df1.head()

,Area,Year,Item1,Value1
258437,Afghanistan,1961,"Meat indigenous, cattle",42984.0
258438,Afghanistan,1962,"Meat indigenous, cattle",45811.0
258439,Afghanistan,1963,"Meat indigenous, cattle",47243.0
258440,Afghanistan,1964,"Meat indigenous, cattle",47999.0
258441,Afghanistan,1965,"Meat indigenous, cattle",48715.0


In [412]:
df2 = select_on(dfs_prod_animals["production"],
                  column_filter_dict={"Item" : ['Meat, cattle'
                                                #,'Meat, chicken', 'Meat, pig'
                                               ], "Unit":["tonnes"]}).sort_values(by=["Area","Year","Item"])[["Area","Year","Item","Value"]]
df2.columns = [["Area","Year","Item2","Value2"]]
df2.head()

,Area,Year,Item2,Value2
539,Afghanistan,1961,"Meat, cattle",43000.0
540,Afghanistan,1962,"Meat, cattle",45800.0
541,Afghanistan,1963,"Meat, cattle",47250.0
542,Afghanistan,1964,"Meat, cattle",48000.0
543,Afghanistan,1965,"Meat, cattle",48700.0


In [416]:
df = df1.merge(df2)
display(df)


df["diff"] = df["Value1"] - df["Value2"]

,Area,Year,Item1,Value1,Item2,Value2
0,Afghanistan,1961,"Meat indigenous, cattle",42984.0,"Meat, cattle",43000.0
1,Afghanistan,1962,"Meat indigenous, cattle",45811.0,"Meat, cattle",45800.0
2,Afghanistan,1963,"Meat indigenous, cattle",47243.0,"Meat, cattle",47250.0
3,Afghanistan,1964,"Meat indigenous, cattle",47999.0,"Meat, cattle",48000.0
4,Afghanistan,1965,"Meat indigenous, cattle",48715.0,"Meat, cattle",48700.0
5,Afghanistan,1966,"Meat indigenous, cattle",68010.0,"Meat, cattle",68000.0
6,Afghanistan,1967,"Meat indigenous, cattle",65008.0,"Meat, cattle",65000.0
7,Afghanistan,1968,"Meat indigenous, cattle",71019.0,"Meat, cattle",71000.0
8,Afghanistan,1969,"Meat indigenous, cattle",75019.0,"Meat, cattle",75000.0
9,Afghanistan,1970,"Meat indigenous, cattle",61970.0,"Meat, cattle",62000.0


,Value1,Value2
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [379]:
display(select_on(dfs_prod_animals["production"],
                  column_filter_dict={"Item" : ['Meat indigenous, cattle',
       'Meat indigenous, chicken', 'Meat indigenous, pig','Meat, cattle',
       'Meat, chicken', 'Meat, pig'], "Unit":["tonnes"]})
        .sort_values(by=["Area","Year","Item"]))

,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
258437,2,Afghanistan,944,"Meat indigenous, cattle",5510,1961,tonnes,42984.0,Calculated data
258543,2,Afghanistan,1094,"Meat indigenous, chicken",5510,1961,tonnes,5600.0,Calculated data
539,2,Afghanistan,867,"Meat, cattle",5510,1961,tonnes,43000.0,FAO estimate
258999,2,Afghanistan,1058,"Meat, chicken",5510,1961,tonnes,5600.0,Calculated data
258438,2,Afghanistan,944,"Meat indigenous, cattle",5510,1962,tonnes,45811.0,Calculated data
258544,2,Afghanistan,1094,"Meat indigenous, chicken",5510,1962,tonnes,6000.0,Calculated data
540,2,Afghanistan,867,"Meat, cattle",5510,1962,tonnes,45800.0,FAO estimate
259000,2,Afghanistan,1058,"Meat, chicken",5510,1962,tonnes,6000.0,Calculated data
258439,2,Afghanistan,944,"Meat indigenous, cattle",5510,1963,tonnes,47243.0,Calculated data
258545,2,Afghanistan,1094,"Meat indigenous, chicken",5510,1963,tonnes,6160.0,Calculated data


In [45]:
display(select_on(dfs_prod_animals["production"],
                  column_filter_dict={"Item" : ['Eggs, hen, in shell',
'Eggs, hen, in shell (number)',
'Eggs, other bird, in shell',
'Eggs, other bird, in shell (number)',
'Eggs Primary'], "Unit":["tonnes"],"Area":["China, mainland"]})
        .groupby(by=["Area","Year","Unit","Item"]).sum().head(100))

Area Code  Item Code  \
Area            Year Unit   Item                                               
China, mainland 1961 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1962 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1963 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1964 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1965 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1966 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1967 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1968 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1969 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1970 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
...                                                           ...        ...   
                1984 tonnes Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1985 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1986 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1987 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1988 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1989 tonnes Eggs Primary                       41       1783   
                            Eggs, hen, in shell                41       1062   
                            Eggs, other bird, in shell         41       1091   
                1990 tonnes Eggs Primary                

In [143]:
print(dfs_prod_animals["producing_animals/slaughtered"]["Item"].unique())
dfs_prod_animals["producing_animals/slaughtered"].head()

['Hides, cattle, fresh' 'Meat, camel' 'Meat, cattle' 'Meat, chicken'
 'Meat, goat' 'Meat, sheep' 'Skins, goat, fresh' 'Skins, sheep, fresh'
 'Meat, pig' 'Meat, horse' 'Meat, rabbit' 'Meat, turkey' 'Meat, duck'
 'Meat, goose and guinea fowl' 'Skins, sheep, with wool'
 'Hides, buffalo, fresh' 'Meat, buffalo' 'Meat, other camelids'
 'Meat, other rodents' 'Meat, ass' 'Meat, mule' 'Meat, bird nes'
 'Meat, game' 'Meat, nes' 'Beef and Buffalo Meat' 'Meat, Poultry'
 'Sheep and Goat Meat']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
145,2,Afghanistan,919,"Hides, cattle, fresh",5320,1961,Head,360000.0,FAO estimate
146,2,Afghanistan,919,"Hides, cattle, fresh",5320,1962,Head,384000.0,FAO estimate
147,2,Afghanistan,919,"Hides, cattle, fresh",5320,1963,Head,396000.0,FAO estimate
148,2,Afghanistan,919,"Hides, cattle, fresh",5320,1964,Head,402000.0,FAO estimate
149,2,Afghanistan,919,"Hides, cattle, fresh",5320,1965,Head,408000.0,FAO estimate


Area Code  Item Code  Element Code  \
Unit Item                                                              
Head Beef and Buffalo Meat          1324237   18961194      55854680   
     Hides, buffalo, fresh           198485    1521630       8458800   
     Hides, cattle, fresh           1291748    9431697      54599160   
     Meat, Poultry                  1335816   19034624      56019488   
     Meat, ass                        79758     538488       2585520   
     Meat, bird nes                   75613     628353       3070217   
     Meat, buffalo                   226561    1660091       9325960   
     Meat, camel                     280945    2146935      10134600   
     Meat, cattle                   1324588    9103500      55860000   
     Meat, chicken                  1336167   11139682      56024809   
     Meat, duck                      526714    4526146      22529114   
     Meat, game                         477      61639        281960   
     Meat, goat                     1181181    9503865      49715400   
     Meat, goose and guinea fowl     279852    2368111      11743447   
     Meat, horse                     507127    4307919      20891640   
     Meat, mule                       32130     179982        861840   
     Meat, nes                         3604      61798        281960   
     Meat, other camelids             10206     125064        574560   
     Meat, other rodents              10206     124308        574668   
     Meat, pig                      1211199    9947385      51130520   
     Meat, rabbit                    353989    3336284      15558604   
     Meat, sheep                    1203382    9414372      51263520   
     Meat, turkey                    422999    3748680      18469191   
     Sheep and Goat Meat            1306962   18626556      54838560   
     Skins, goat, fresh             1131676    9228075      47895960   
     Skins, sheep, fresh            1168271    9315190      49805840   
     Skins, sheep, with wool         281034    2250747      11985960   

                                      Year         Value  
Unit Item                                                 
Head Beef and Buffalo Meat        20874815  1.512987e+10  
     Hides, buffalo, fresh         3159330  1.247652e+09  
     Hides, cattle, fresh         20399793  1.450160e+10  
     Meat, Poultry                20932090  1.869302e+12  
     Meat, ass                      965925  1.222840e+08  
     Meat, bird nes                1147331  2.269256e+09  
     Meat, buffalo                 3484450  9.475568e+08  
     Meat, camel                   3788060  7.679914e+07  
     Meat, cattle                 20876805  1.419162e+10  
     Meat, chicken                20934080  1.715423e+12  
     Meat, duck                    8417906  1.039037e+11  
     Meat, game                     105311  0.000000e+00  
     Meat, goat                   18580005  1.551745e+10  
     Meat, goose and guinea fowl   4387872  2.651584e+10  
     Meat, horse                   7810560  2.396207e+08  
     Meat, mule                     321975  3.379057e+07  
     Meat, nes                      105311  0.000000e+00  
     Meat, other camelids           214650  2.985115e+07  
     Meat, other rodents            214650  4.174862e+09  
     Meat, pig                    19109489  6.569652e+10  
     Meat, rabbit                  5816545  4.088878e+10  
     Meat, sheep                  19159605  2.642682e+10  
     Meat, turkey                  6901150  2.390748e+10  
     Sheep and Goat Meat          20495565  4.185548e+10  
     Skins, goat, fresh           17893017  1.612604e+10  
     Skins, sheep, fresh          18609506  2.633272e+10  
     Skins, sheep, with wool       4481707  3.090197e+07

In [148]:
print(dfs_prod_animals["milk_animals"]["Item"].unique())
dfs_prod_animals["milk_animals"].head()

['Milk, whole fresh camel' 'Milk, whole fresh cow'
 'Milk, whole fresh goat' 'Milk, whole fresh sheep'
 'Milk, whole fresh buffalo' 'Milk,Total']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
703,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1961,Head,25000.0,FAO estimate
704,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1962,Head,28000.0,FAO estimate
705,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1963,Head,35000.0,FAO estimate
706,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1964,Head,33000.0,FAO estimate
707,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1965,Head,30000.0,FAO estimate


In [149]:
print(dfs_prod_animals["prod_popultn"]["Item"].unique())
dfs_prod_animals["prod_popultn"].head()

['Wool, greasy' 'Honey, natural' 'Beeswax' 'Silk-worm cocoons, reelable'
 'Hair, horse']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
1097,2,Afghanistan,987,"Wool, greasy",5319,1961,Head,18000000.0,FAO estimate
1098,2,Afghanistan,987,"Wool, greasy",5319,1962,Head,18150000.0,FAO estimate
1099,2,Afghanistan,987,"Wool, greasy",5319,1963,Head,18500000.0,FAO estimate
1100,2,Afghanistan,987,"Wool, greasy",5319,1964,Head,19000000.0,FAO estimate
1101,2,Afghanistan,987,"Wool, greasy",5319,1965,Head,19600000.0,FAO estimate


In [150]:
print(dfs_prod_animals["milk_animals"]["Item"].unique())
dfs_prod_animals["milk_animals"].head()

['Milk, whole fresh camel' 'Milk, whole fresh cow'
 'Milk, whole fresh goat' 'Milk, whole fresh sheep'
 'Milk, whole fresh buffalo' 'Milk,Total']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
703,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1961,Head,25000.0,FAO estimate
704,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1962,Head,28000.0,FAO estimate
705,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1963,Head,35000.0,FAO estimate
706,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1964,Head,33000.0,FAO estimate
707,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1965,Head,30000.0,FAO estimate


In [147]:
print(dfs_prod_animals["yield"]["Item"].unique())
dfs_prod_animals["yield"].head()

['Eggs, hen, in shell' 'Hides, cattle, fresh' 'Milk, whole fresh camel'
 'Milk, whole fresh cow' 'Milk, whole fresh goat'
 'Milk, whole fresh sheep' 'Skins, goat, fresh' 'Skins, sheep, fresh'
 'Wool, greasy' 'Eggs Primary' 'Milk,Total' 'Milk, whole fresh buffalo'
 'Skins, sheep, with wool' 'Eggs, other bird, in shell'
 'Hides, buffalo, fresh' 'Honey, natural' 'Beeswax']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
258118,2,Afghanistan,1062,"Eggs, hen, in shell",5410,1961,g/An,2500.0,Calculated data
258119,2,Afghanistan,1062,"Eggs, hen, in shell",5410,1962,g/An,2500.0,Calculated data
258120,2,Afghanistan,1062,"Eggs, hen, in shell",5410,1963,g/An,2500.0,Calculated data
258121,2,Afghanistan,1062,"Eggs, hen, in shell",5410,1964,g/An,2500.0,Calculated data
258122,2,Afghanistan,1062,"Eggs, hen, in shell",5410,1965,g/An,2461.5,Calculated data


In [135]:
print(dfs_prod_animals["yield/carcass_weight"]["Item"].unique())
dfs_prod_animals["yield/carcass_weight"].head()

['Meat indigenous, camel' 'Meat indigenous, cattle'
 'Meat indigenous, chicken' 'Meat indigenous, goat'
 'Meat indigenous, sheep' 'Meat, camel' 'Meat, cattle' 'Meat, chicken'
 'Meat, goat' 'Meat, sheep' 'Beef and Buffalo Meat' 'Meat, Poultry'
 'Sheep and Goat Meat' 'Meat indigenous, pig' 'Meat, pig'
 'Meat indigenous, horse' 'Meat indigenous, rabbit'
 'Meat indigenous, turkey' 'Meat, horse' 'Meat, rabbit' 'Meat, turkey'
 'Meat indigenous, duck' 'Meat indigenous, geese' 'Meat, duck'
 'Meat, goose and guinea fowl' 'Meat indigenous, buffalo' 'Meat, buffalo'
 'Meat indigenous, other camelids' 'Meat indigenous, rodents'
 'Meat, other camelids' 'Meat, other rodents' 'Meat indigenous, ass'
 'Meat, ass' 'Meat indigenous, mule' 'Meat, bird nes' 'Meat, mule'
 'Meat indigenous, bird nes']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
258278,2,Afghanistan,1137,"Meat indigenous, camel",5417,1961,kg/An,180.0,Calculated data
258279,2,Afghanistan,1137,"Meat indigenous, camel",5417,1962,kg/An,180.0,Calculated data
258280,2,Afghanistan,1137,"Meat indigenous, camel",5417,1963,kg/An,180.0,Calculated data
258281,2,Afghanistan,1137,"Meat indigenous, camel",5417,1964,kg/An,180.0,Calculated data
258282,2,Afghanistan,1137,"Meat indigenous, camel",5417,1965,kg/An,180.0,Calculated data


In [ ]:
# Harmonise units
## 1000 Head -> Head
unit_filter = trade_matrix_live_animals["Unit Importer Report"] == "1000 Head"
trade_matrix_live_animals.loc[unit_filter, "Quantity Importer Report"] = trade_matrix_live_animals[unit_filter]["Quantity Importer Report"].multiply(1000)
trade_matrix_live_animals.loc[unit_filter, "Unit Importer Report"] = "Head"

unit_filter = trade_matrix_live_animals["Unit Exporter Report"] == "1000 Head"
trade_matrix_live_animals.loc[unit_filter, "Quantity Exporter Report"] = trade_matrix_live_animals[unit_filter]["Quantity Exporter Report"].multiply(1000)
trade_matrix_live_animals.loc[unit_filter, "Unit Exporter Report"] = "Head"

## TODO 
## tonnes -> Head ???